In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import gta.vision

In [ ]:
reload(gta.vision)
win = gta.vision.GtaWindow()
img = win.grab()

In [ ]:
np.array(img).shape

In [ ]:
recorder = gta.vision.VisionRecorder()

In [ ]:
recorder.stop()

In [ ]:
plt.imshow(recorder.results[-1])